<a href="https://colab.research.google.com/github/aishwarya-ar1/HPE-Deep-Learning/blob/main/Handwritten_Text_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
print(tf.__version__)

import cv2
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from PIL import Image
import math
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
directory='/content/forms'

2.4.1


In [3]:
def image_binarization(image):
  # converting image to grayscale
  image_grayscale=image.convert('L')
  img=np.array(image_grayscale) 
   
  img[img <128] = 0
  img[img >=128] = 254 

  #making white as foreground pixels and black as background pixels
  img[img==0]=255
  img[img==254]=0
  
  return img


def line_segmentation(img):
  start_matrix=[]
  end_matrix=[]

  #matrix to get the start and end points of a line
  
  lines=[]
  begin_matrix=[]
  stop_matrix=[]
  del_start_matrix=[]
  del_end_matrix=[]

  horizontal_hist = np.sum(img,axis=1,keepdims=True)/255
  start_count=0

  for i in range(len(horizontal_hist)):

    if horizontal_hist[i]>0 and horizontal_hist[i-1]==0:
      start_count+=1
      start_matrix.append(i)

    if horizontal_hist[i]==0 and start_count>0 and horizontal_hist[i-1]>0:
      end_matrix.append(i)

  
  
  if len(start_matrix)==len(end_matrix):
    for i in range(len(start_matrix)):
      if end_matrix[i]-start_matrix[i]<20:
        del_start_matrix.append(i)
        del_end_matrix.append(i)
    for i in range(len(start_matrix)):
      count=0
      for j in range(len(del_start_matrix)):
        if i==del_start_matrix[j]:
          count=count+1
      if count==0:
        begin_matrix.append(start_matrix[i])
    for i in range(len(end_matrix)):
      count=0
      for j in range(len(del_end_matrix)):
        if i==del_end_matrix[j]:
          count=count+1
      if count==0:
        stop_matrix.append(end_matrix[i])
    
    for i in range(len(begin_matrix)):
      lines.append(img[begin_matrix[i]:stop_matrix[i],:])

      

      

  
  return lines




def word_segmentation(img):
  start_matrix=[]
  end_matrix=[]

  #matrix to get the start and end points of a word
  dissection_matrix=[]
  
  words=[]
  m,n=img.shape
  
  length=[]
  vertical_hist = np.sum(img,axis=0,keepdims=True)/255
  
  
  start_count=0
  
  
  for i in range(len(vertical_hist[0])):
    if vertical_hist[0][i]>0 and vertical_hist[0][i-1]==0:
      start_count+=1
      start_matrix.append(i)
    if vertical_hist[0][i]==0 and start_count>0 and vertical_hist[0][i-1]>0:
      end_matrix.append(i)
  
  
  length_mag=0
  for i in range(len(start_matrix)):
    if i>0:
      length_mag=(start_matrix[i]-end_matrix[i-1])
      length.append(length_mag)

  max=np.max(length)
     
  avg=max/3

  dissection_matrix.append([start_matrix[0],end_matrix[0]])
  j=0
  for i in range(len(length)-1):
    
    if length[i]> avg:
      dissection_matrix.append([start_matrix[i+1],end_matrix[i+1]])
      j=j+1
      
    if length[i]<=avg:
      dissection_matrix[j][1]=end_matrix[i+1]
  for i in range(len(dissection_matrix)):
    words.append(img[0:m,dissection_matrix[i][0]:dissection_matrix[i][1]])
  
  return words


def char_segmentation(img):
  start_matrix=[]
  dissection_matrix=[]
  delete_matrix=[]
  address_matrix=[]
  
  
  characters=[]
  m,n=img.shape

  vertical_hist = np.sum(img,axis=0,keepdims=True)/255
  start_matrix.append(0)

  for i in range(len(vertical_hist[0])):
    if vertical_hist[0][i]<10:
      start_matrix.append(i)

  for i in range(len(start_matrix)-1):
    if start_matrix[i+1]-start_matrix[i]<10:
      delete_matrix.append(i)
  
  for i in range(len(start_matrix)):
    count=0
    for j in range(len(delete_matrix)):
      if  i==delete_matrix[j]:
        count=count+1
    if count==0:
      address_matrix.append(start_matrix[i])


  for i in range(len(address_matrix)-1):
    dissection_matrix.append([address_matrix[i],address_matrix[i+1]])
  for i in range(len(dissection_matrix)):
    characters.append(img[0:m,dissection_matrix[i][0]:dissection_matrix[i][1]])
    
    
  return characters


In [4]:
def data_label():

  samples=[]
  labels=[]

 
  
 
  
  
  
  for filename in os.listdir(directory):
    count=0
    text=open('/content/lines_new.txt')
    image_orig=Image.open('/content/forms/'+filename,'r')
    img=image_binarization(image_orig)
    lines=line_segmentation(img)
    for line in text:
      line_Split = line.strip().split(" ")
      filename_split=filename.split('.')
      linesplit=line_Split[0].split("-")
      string= linesplit[0]+'-'+linesplit[1]
      if string==filename_split[0]:
          count=count+1
    if count==len(lines): 
      for i in range(len(lines)):
        try:
          words=word_segmentation(lines[i])
          text=open('/content/lines_new.txt')
          for line in text:
            lineSplit = line.strip().split(' ')
            filename_split=filename.split('.')
            if lineSplit[0]==filename_split[0]+'-'+str(0)+str(i):
              word_split = lineSplit[8].split('|')
              if len(word_split)==len(words):
                for j in range(len(words)):
                  characters=char_segmentation(words[j])
                  if len(word_split[j])==len(characters):
                    for k in range(len(characters)):
                      
                      character=cv2.resize(characters[k],(96,96))
                      samples.append(character)
                    for l in word_split[j]:
                      labels.append(l) 

        except ValueError as ve:
          continue  
          
    
  
                 
  
  return  samples,labels

samples,labels=data_label()
print(len(samples))
print(len(labels))
for i in range(5):
  print(labels[i])


149
149
M
r
.
l
a


In [5]:
samples = np.array(samples, dtype='float' )/ 255.0
labels = np.array(labels)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
labels
samples.shape


(149, 96, 96)

In [6]:
from sklearn.model_selection import train_test_split
trainx,testx,trainy,testy = train_test_split(samples, labels, test_size=0.2, random_state=42)


In [7]:
len(testx)
testx.shape
trainx=trainx.reshape(trainx.shape[0],96,96,1)
testx=testx.reshape(testx.shape[0],96,96,1)

In [8]:
print(len(lb.classes_)) 
testy.shape 
testx.shape

28


(30, 96, 96, 1)

In [9]:
HP_model_path = 'bin/model'
HP_binarized_labels = 'bin/labels'
HP_metrics_storage = 'eval'


In [10]:
pip install keras-tuner==1.0.0

     |████████████████████████████████| 92kB 6.0MB/s 
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp37-none-any.whl size=15356 sha256=ccd7a048c902e289b689152c2f3cef22c024f6322e89f9bf70e9a5e06bce5494
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built terminaltables


In [11]:
from kerastuner.tuners import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [13]:

from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Activation,Dropout

In [14]:
classes = len(lb.classes_)

In [15]:

HP_img_dims = (96,96,1)
def build(hp):
    classes = len(lb.classes_)
    model = keras.models.Sequential()
    # BLOCK1
    model.add(Conv2D(hp.Int('input_units',32,512,32),(3,3), 
                     input_shape=trainx.shape[1:]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(hp.Choice('dropout',[0.25,0.4,0.5])))
    

    for i in range(hp.Int("n_layers",1,6)):
      model.add(Conv2D(hp.Int(f'conv_{i}_units',32,512,32),(3,3)))
      model.add(Activation('relu'))
      model.add(Dropout(hp.Choice(f'dropout_{i}',[0.25,0.4,0.5])))

     
   
   

  

    # BLOCK 5- Image Classification (OBJECT)
    model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

    model.add(Dense(classes))
    model.add(Activation("softmax"))
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',[1e-2,1e-3,1e-4])),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
    

    

    return model
trainx=trainx.reshape(trainx.shape[0],96,96,1)
testx=testx.reshape(testx.shape[0],96,96,1)

In [16]:
tuner=RandomSearch(
    build,objective='val_accuracy',
    max_trials=15,executions_per_trial=1)

tuner.search(x=trainx,
             y=trainy,
             epochs=5,
             batch_size=64,
             validation_data=(testx,testy))

Epoch 1/5
2/2 [==============================] - 16s 8s/step - loss: 3.2374 - accuracy: 0.0612 - val_loss: 3.2830 - val_accuracy: 0.0667
Epoch 2/5
2/2 [==============================] - 15s 7s/step - loss: 2.5026 - accuracy: 0.2242 - val_loss: 3.3218 - val_accuracy: 0.1333
Epoch 3/5
2/2 [==============================] - 15s 7s/step - loss: 2.1598 - accuracy: 0.4039 - val_loss: 3.9124 - val_accuracy: 0.0667
Epoch 4/5
2/2 [==============================] - 15s 7s/step - loss: 1.6186 - accuracy: 0.5080 - val_loss: 6.0299 - val_accuracy: 0.1000
Epoch 5/5
2/2 [==============================] - 15s 7s/step - loss: 1.0311 - accuracy: 0.6762 - val_loss: 7.9130 - val_accuracy: 0.1333


Epoch 1/5
2/2 [==============================] - 40s 20s/step - loss: 403.1648 - accuracy: 0.0492 - val_loss: 3.3158 - val_accuracy: 0.0667
Epoch 2/5
2/2 [==============================] - 39s 19s/step - loss: 3.2293 - accuracy: 0.1477 - val_loss: 5.7681 - val_accuracy: 0.0667
Epoch 3/5
2/2 [==============================] - 39s 19s/step - loss: 3.9489 - accuracy: 0.1633 - val_loss: 3.2714 - val_accuracy: 0.2000
Epoch 4/5
2/2 [==============================] - 39s 19s/step - loss: 3.1510 - accuracy: 0.1201 - val_loss: 3.4104 - val_accuracy: 0.2000
Epoch 5/5
2/2 [==============================] - 39s 19s/step - loss: 2.9012 - accuracy: 0.1645 - val_loss: 3.3007 - val_accuracy: 0.0667


Epoch 1/5
2/2 [==============================] - 66s 32s/step - loss: 3.2530 - accuracy: 0.1049 - val_loss: 13.3739 - val_accuracy: 0.0667
Epoch 2/5
2/2 [==============================] - 75s 32s/step - loss: 8.4950 - accuracy: 0.1850 - val_loss: 3.2463 - val_accuracy: 0.1667
Epoch 3/5
2/2 [==============================] - 64s 32s/step - loss: 3.0083 - accuracy: 0.1805 - val_loss: 3.2052 - val_accuracy: 0.1333
Epoch 4/5
2/2 [==============================] - 64s 32s/step - loss: 2.8848 - accuracy: 0.1970 - val_loss: 3.3352 - val_accuracy: 0.1333
Epoch 5/5
2/2 [==============================] - 65s 32s/step - loss: 2.7403 - accuracy: 0.2302 - val_loss: 3.2554 - val_accuracy: 0.1000


Epoch 1/5
2/2 [==============================] - 12s 6s/step - loss: 3.3310 - accuracy: 0.0220 - val_loss: 3.3130 - val_accuracy: 0.1000
Epoch 2/5
2/2 [==============================] - 12s 6s/step - loss: 3.2613 - accuracy: 0.1973 - val_loss: 3.2810 - val_accuracy: 0.1000
Epoch 3/5
2/2 [==============================] - 11s 6s/step - loss: 3.1568 - accuracy: 0.2082 - val_loss: 3.2358 - val_accuracy: 0.0667
Epoch 4/5
2/2 [==============================] - 11s 6s/step - loss: 3.0043 - accuracy: 0.2506 - val_loss: 3.1916 - val_accuracy: 0.0667
Epoch 5/5
2/2 [==============================] - 12s 6s/step - loss: 2.8427 - accuracy: 0.2350 - val_loss: 3.1801 - val_accuracy: 0.0667


Epoch 1/5
2/2 [==============================] - 43s 21s/step - loss: 7490.1317 - accuracy: 0.0720 - val_loss: 3.2216 - val_accuracy: 0.2000
Epoch 2/5
2/2 [==============================] - 43s 21s/step - loss: 3.1248 - accuracy: 0.1197 - val_loss: 4.4679 - val_accuracy: 0.0667
Epoch 3/5
2/2 [==============================] - 42s 21s/step - loss: 3.4374 - accuracy: 0.1425 - val_loss: 3.3248 - val_accuracy: 0.0333
Epoch 4/5
2/2 [==============================] - 43s 21s/step - loss: 3.2701 - accuracy: 0.0921 - val_loss: 3.3380 - val_accuracy: 0.0333
Epoch 5/5
2/2 [==============================] - 42s 21s/step - loss: 3.2426 - accuracy: 0.1529 - val_loss: 3.3559 - val_accuracy: 0.0667


...

Epoch 1/5
2/2 [==============================] - 58s 28s/step - loss: 3.3023 - accuracy: 0.0668 - val_loss: 3.2616 - val_accuracy: 0.1000
Epoch 2/5
2/2 [==============================] - 57s 28s/step - loss: 2.9208 - accuracy: 0.1705 - val_loss: 3.1791 - val_accuracy: 0.0667
Epoch 3/5
2/2 [==============================] - 57s 28s/step - loss: 2.6493 - accuracy: 0.2406 - val_loss: 3.2833 - val_accuracy: 0.0667
Epoch 4/5
2/2 [==============================] - 57s 28s/step - loss: 2.5099 - accuracy: 0.2358 - val_loss: 3.2097 - val_accuracy: 0.0333
Epoch 5/5
2/2 [==============================] - 57s 28s/step - loss: 2.3190 - accuracy: 0.3447 - val_loss: 3.2171 - val_accuracy: 0.0667


Epoch 1/5
2/2 [==============================] - 39s 19s/step - loss: 3.2987 - accuracy: 0.0716 - val_loss: 3.2446 - val_accuracy: 0.0667
Epoch 2/5
2/2 [==============================] - 39s 19s/step - loss: 2.8781 - accuracy: 0.2078 - val_loss: 3.2261 - val_accuracy: 0.0667
Epoch 3/5
2/2 [==============================] - 38s 19s/step - loss: 2.5497 - accuracy: 0.2242 - val_loss: 3.5048 - val_accuracy: 0.0667
Epoch 4/5
2/2 [==============================] - 46s 19s/step - loss: 2.4397 - accuracy: 0.2630 - val_loss: 3.4163 - val_accuracy: 0.0333
Epoch 5/5
2/2 [==============================] - 38s 19s/step - loss: 2.2198 - accuracy: 0.3462 - val_loss: 3.4439 - val_accuracy: 0.0667


Epoch 1/5
2/2 [==============================] - 17s 8s/step - loss: 3.2421 - accuracy: 0.0728 - val_loss: 5.3462 - val_accuracy: 0.0667
Epoch 2/5
2/2 [==============================] - 16s 8s/step - loss: 3.5297 - accuracy: 0.1861 - val_loss: 3.2253 - val_accuracy: 0.0333
Epoch 3/5
2/2 [==============================] - 16s 8s/step - loss: 2.7694 - accuracy: 0.2894 - val_loss: 3.2300 - val_accuracy: 0.0667
Epoch 4/5
2/2 [==============================] - 17s 8s/step - loss: 2.6875 - accuracy: 0.3979 - val_loss: 3.5212 - val_accuracy: 0.0667
Epoch 5/5
2/2 [==============================] - 17s 8s/step - loss: 2.3471 - accuracy: 0.3663 - val_loss: 4.8671 - val_accuracy: 0.0667


Epoch 1/5
2/2 [==============================] - 23s 11s/step - loss: 3.2656 - accuracy: 0.0668 - val_loss: 3.2122 - val_accuracy: 0.1333
Epoch 2/5
2/2 [==============================] - 22s 11s/step - loss: 2.6600 - accuracy: 0.2842 - val_loss: 4.1030 - val_accuracy: 0.1333
Epoch 3/5
2/2 [==============================] - 21s 11s/step - loss: 2.1797 - accuracy: 0.3279 - val_loss: 3.5364 - val_accuracy: 0.1333
Epoch 4/5
2/2 [==============================] - 22s 11s/step - loss: 1.6685 - accuracy: 0.6173 - val_loss: 4.4584 - val_accuracy: 0.1000
Epoch 5/5
2/2 [==============================] - 22s 11s/step - loss: 1.0756 - accuracy: 0.6493 - val_loss: 7.1235 - val_accuracy: 0.1000


INFO:tensorflow:Oracle triggered exit


In [19]:

HP_epoch = 1000
batch_size=16

HP_image_dims = (96,96,1)


from keras.layers.core import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential
from keras import backend 
import keras
import keras.utils
from keras import utils as np_utils
from keras.optimizers import Adam
from sklearn import metrics
from tensorflow.keras.callbacks import EarlyStopping
import statistics




HP_img_dims = (192,192,1)
HP_small_pattern = (2,2) # UNCOMPRESSED or 1-2 compression IMAGES
HP_large_pattern = (3,3)

In [39]:

class RacoonVGG:
  @staticmethod
  def build(height, width,depth, classes,dropout, neuronPct, neuronShrink,lr_rate):
    input_shape = (height, width, depth)
    channel_dim = -1 # last position 
    if backend.image_data_format() == 'channels_first':
      input_shape = (depth, height, width)
      channel_dim = 1
    
    small_pattern = (2,2) # UNCOMPRESSED or 1-2 compression IMAGES
    large_pattern = (3,3)
    model = Sequential()
    neuronCount = int(neuronPct * 5000)
    layer = 0
    count=0
    while neuronCount>25 and layer<10:
        # The first (0th) layer needs an input input_dim(neuronCount)
        if layer==0:
          model.add(Conv2D(int(neuronCount),HP_small_pattern, padding='same',
                     input_shape=input_shape))
          model.add(Activation('relu'))
          model.add(BatchNormalization(axis=channel_dim))
          model.add(MaxPooling2D(pool_size=HP_small_pattern))
          model.add(Dropout(dropout))
          neuronCount = neuronCount * neuronShrink
          layer += 1

        
        if layer>0 and layer<10:
          model.add(Conv2D(int(neuronCount),HP_small_pattern, padding='same'))
          model.add(Activation('relu'))
          model.add(BatchNormalization(axis=channel_dim))
          model.add(MaxPooling2D(pool_size=HP_large_pattern))
          model.add(Dropout(dropout))
          layer += 1
          neuronCount = neuronCount * neuronShrink
        
          
        
    




            
    model.add(Flatten())
    model.add(Dense(int(neuronCount)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(int(neuronCount)))
    model.add(Activation('relu'))
    model.add(BatchNormalization())
    model.add(Dropout(dropout))
    model.add(Dense(classes))
    model.add(Activation('softmax'))
    return model  

    
  

    

#trainx=trainx.reshape(trainx.shape[0],192,192,1)
#testx=testx.reshape(testx.shape[0],192,192,1)

In [33]:
def evaluate_network(dropout, neuronPct, neuronShrink,lr_rate):
  epochs_needed = []
  mean_benchmark = []

  model = RacoonVGG.build(height=HP_img_dims[0], width=HP_img_dims[1],depth=HP_img_dims[2],classes = len(lb.classes_),
                          dropout=dropout, neuronPct=neuronPct, neuronShrink=neuronShrink,lr_rate=lr_rate
                          ) 
  opt = keras.optimizers.Adam(learning_rate=lr_rate)
  model.compile(loss='categorical_crossentropy', optimizer=opt)
  monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, 
        patience=100, verbose=0, mode='auto', restore_best_weights=True)
  hist = model.fit(aug.flow(trainx, trainy, batch_size=batch_size),
                 validation_data=(testx, testy),verbose=0,
                 steps_per_epoch= len(trainx) // batch_size,
                  epochs=5)
  epochs = monitor.stopped_epoch
  epochs_needed.append(epochs)
  
  
  score = model.evaluate(testx, testy)
  acc=(1/score)
  
  #mean_benchmark.append(score)
  #m1 = statistics.mean(mean_benchmark)
  keras.backend.clear_session()
  
  return acc

In [30]:
!pip install bayesian-optimization
from bayes_opt import BayesianOptimization

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp37-none-any.whl size=11687 sha256=6a4bb02384b7dbc51e5a49cd66764668e55e5d035e8eb72311bc13ed86880c32
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


In [35]:
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)

pbounds = {'dropout': (0.0, 0.499),
           'neuronPct': (0.01, 1),
           'neuronShrink': (0.01, 1),
           'lr_rate': (0.0, 0.1),
          }

optimizer = BayesianOptimization(
    f=evaluate_network,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum 
    # is observed, verbose = 0 is silent
    random_state=1)

optimizer.maximize(init_points=4, n_iter=2)

print(optimizer.max)

|   iter    |  target   |  dropout  |  lr_rate  | neuronPct | neuron... |
-------------------------------------------------------------------------


NameError: ignored